In [1]:
from fuzzywuzzy import fuzz
import pandas as pd

/usr/local/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv("../../data/cook_county.csv")

In [3]:
df.head()

,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,County of Purchase,Acres,Price per Acre,Total Price,Type of Sale,Date of Purchase,Volume,Page
0,ABBE CHRISTIAN,UNKNOWN,A,LOT2NWNE,16.0,36N,13E,3.0,COOK,40.02,4.0,160.08,SC,09/01/1852,819,64.0
1,ABBOTT ALLISON,UNKNOWN,A,LOT1SW,6.0,40N,13E,3.0,COOK,79.72,1.25,99.65,FD,02/17/1841,687,191.0
2,ABBOTT JOSEPH,UNKNOWN,NaN,NWSW,3.0,37N,12E,3.0,COOK,40.0,9.0,360.00,CN,05/09/1853,L7A,180.0
3,ABBOTT SAMUEL,UNKNOWN,NaN,SW,14.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/27/1838,687,194.0
4,ABBOTT SAMUEL S,UNKNOWN,NaN,SE,15.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/17/1838,687,194.0


In [4]:
len(df)

13274

In [5]:
df2 = df.drop_duplicates()

In [7]:
attr = pd.read_csv("../../data/sections.csv")

In [8]:
attr.head()

,OBJECTID,MERIDIAN,SECTION,TWPNUM,RNGNUM,RNGDIR,TWPDIR
0,3397,3,5,42,9,E,N
1,3404,3,1,42,9,E,N
2,3406,3,6,42,10,E,N
3,3409,3,5,42,10,E,N
4,3420,3,2,42,10,E,N


In [63]:
def get_object_id(row):
    if pd.isnull(row['Township']):
        return None
    twp = int(row['Township'][:-1])
    twp_dir = row['Township'][-1]
    rng = int(row['Range'][:-1])
    rng_dir = row['Range'][-1]
    ids = attr.loc[(attr["MERIDIAN"] == row['Meridian'])\
             & (attr["SECTION"] == row['Section Number'])\
             & (attr["TWPNUM"] == twp)\
             & (attr['RNGNUM'] == rng)\
             & (attr['TWPDIR'] == twp_dir)\
             & (attr['RNGDIR'] == rng_dir), 'OBJECTID'].values
    if len(ids) < 1:
        return None
    
    return ids[0]

In [64]:
object_ids = df2.apply(get_object_id, axis=1)

In [65]:
df2['Shapefile Object ID'] = object_ids

<ipython-input-65-277c87abb599>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Shapefile Object ID'] = object_ids


In [66]:
df2['Voided'] = (df2['Aliquot Parts or Lot'].str.contains("VO")) | (df2['Aliquot Parts or Lot'].str[-1] == 'V')

<ipython-input-66-5a669d4867a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Voided'] = (df2['Aliquot Parts or Lot'].str.contains("VO")) | (df2['Aliquot Parts or Lot'].str[-1] == 'V')


In [67]:
df2[df2['Voided']]

,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,County of Purchase,Acres,Price per Acre,Total Price,Type of Sale,Date of Purchase,Volume,Page,Shapefile Object ID,Voided
362,BAILEY AMOS,UNKNOWN,NaN,L5BL16CHIV,NaN,NaN,NaN,NaN,COOK,0.0,000.00,2000.00,CN,06/30/1836,L5A,28.0,NaN,True
442,BARCKENBILE CHRISTIA,UNKNOWN,A,LOT3BL4VOID,16.0,39N,14E,3.0,COOK,3.25,25.23,82.00,SC,10/23/1833,818,10.0,5909.0,True
529,BEAUBIEN J B,ILLINOIS,I,NWFRPTNWVO,29.0,39N,14E,3.0,COOK,127.66,5.00,638.30,CN,09/29/1830,R13,5.0,6186.0,True
530,BEAUBIEN J B,ILLINOIS,I,NFR2NEVO,29.0,39N,14E,3.0,COOK,84.95,5.00,424.90,CN,09/29/1830,R13,5.0,6186.0,True
628,BENNETT WILLIAM,COOK,NaN,E2NWVOID,20.0,39N,14E,3.0,COOK,80.0,1.25,100.00,FD,10/16/1834,236,39.0,6049.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13213,WRIGHT JOHN F,ILLINOIS,NaN,E2SEVO,5.0,39N,14E,3.0,COOK,80.00,1.25,100.00,CN,02/21/1831,R13,7.0,5633.0,True
13227,WRIGHT T G,UNKNOWN,I,L2BL52CHIOTV,NaN,NaN,NaN,NaN,COOK,0000.00,000.00,9200.00,CN,06/23/1836,L5A,20.0,NaN,True
13228,WRIGHT T G,UNKNOWN,I,L1BL52CHIOTV,NaN,NaN,NaN,NaN,COOK,0000.00,000.00,8210.00,CN,06/23/1836,L5A,20.0,NaN,True
13231,WURTS WELCH ET AL,UNKNOWN,NaN,SWSWVOID,7.0,37N,12E,3.0,COOK,40.00,15.00,600.00,CN,09/09/1852,L5B,2.0,7537.0,True


In [68]:
df2.drop_duplicates(subset=["Purchaser", "Aliquot Parts or Lot", 'Section Number', 'Township', 'Range', 'Meridian', 'Voided'], inplace=True)

<ipython-input-68-30c3e82e51d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop_duplicates(subset=["Purchaser", "Aliquot Parts or Lot", 'Section Number', 'Township', 'Range', 'Meridian', 'Voided'], inplace=True)


In [69]:
len(df2)

12286

In [70]:
df2.to_csv("../../data/cook_county_cleaned.csv")

In [71]:
df2.head()

,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,County of Purchase,Acres,Price per Acre,Total Price,Type of Sale,Date of Purchase,Volume,Page,Shapefile Object ID,Voided
0,ABBE CHRISTIAN,UNKNOWN,A,LOT2NWNE,16.0,36N,13E,3.0,COOK,40.02,4.0,160.08,SC,09/01/1852,819,64.0,8570.0,False
1,ABBOTT ALLISON,UNKNOWN,A,LOT1SW,6.0,40N,13E,3.0,COOK,79.72,1.25,99.65,FD,02/17/1841,687,191.0,4828.0,False
2,ABBOTT JOSEPH,UNKNOWN,NaN,NWSW,3.0,37N,12E,3.0,COOK,40.0,9.0,360.00,CN,05/09/1853,L7A,180.0,7367.0,False
3,ABBOTT SAMUEL,UNKNOWN,NaN,SW,14.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/27/1838,687,194.0,5097.0,False
4,ABBOTT SAMUEL S,UNKNOWN,NaN,SE,15.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/17/1838,687,194.0,5100.0,False


In [73]:
duplicated = df2.duplicated(subset=["Aliquot Parts or Lot", "Section Number", \
                                    "Township", "Range", "Meridian"],keep=False)

In [74]:
threshold = 90

for idx, row in df2.loc[duplicated].iterrows():
    candidates = df2.loc[(df2['Township'] == row['Township'])\
                         & (df2['Range'] == row['Range'])\
                        & (df2['Section Number'] == row['Section Number'])\
                        & (df2['Aliquot Parts or Lot'] == row ["Aliquot Parts or Lot"])]
    matches = []
    for idx2, row2 in candidates.iterrows():
        if fuzz.ratio(row['Purchaser'], row2['Purchaser']) >= threshold:
            matches.append(str(idx2))
    
    df2.loc[idx, 'Lot Matches'] = ";".join(matches)

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a co

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

...

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on 

In [75]:
df2.head()

,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,County of Purchase,Acres,Price per Acre,Total Price,Type of Sale,Date of Purchase,Volume,Page,Shapefile Object ID,Voided,Lot Matches
0,ABBE CHRISTIAN,UNKNOWN,A,LOT2NWNE,16.0,36N,13E,3.0,COOK,40.02,4.0,160.08,SC,09/01/1852,819,64.0,8570.0,False,NaN
1,ABBOTT ALLISON,UNKNOWN,A,LOT1SW,6.0,40N,13E,3.0,COOK,79.72,1.25,99.65,FD,02/17/1841,687,191.0,4828.0,False,NaN
2,ABBOTT JOSEPH,UNKNOWN,NaN,NWSW,3.0,37N,12E,3.0,COOK,40.0,9.0,360.00,CN,05/09/1853,L7A,180.0,7367.0,False,2
3,ABBOTT SAMUEL,UNKNOWN,NaN,SW,14.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/27/1838,687,194.0,5097.0,False,NaN
4,ABBOTT SAMUEL S,UNKNOWN,NaN,SE,15.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/17/1838,687,194.0,5100.0,False,NaN


In [76]:
df2['Purchaser'] = df2['Purchaser'].astype(str)
count = 0
for idx, row in df2.iterrows():
    matches = []
    for idx2, row2 in df2.iterrows():
        if fuzz.ratio(row['Purchaser'], row2['Purchaser']) >= threshold:
            matches.append(str(idx2))
    
    df2.loc[idx, 'Name Matches'] = ";".join(matches)
    count = count + 1
    if count % 1000 == 0:
        print(count)

<ipython-input-76-e52049bbf4fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Purchaser'] = df2['Purchaser'].astype(str)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/davidhackett/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [77]:
df2.head()

,Purchaser,Residence,Social Status,Aliquot Parts or Lot,Section Number,Township,Range,Meridian,County of Purchase,Acres,Price per Acre,Total Price,Type of Sale,Date of Purchase,Volume,Page,Shapefile Object ID,Voided,Lot Matches,Name Matches
0,ABBE CHRISTIAN,UNKNOWN,A,LOT2NWNE,16.0,36N,13E,3.0,COOK,40.02,4.0,160.08,SC,09/01/1852,819,64.0,8570.0,False,NaN,0
1,ABBOTT ALLISON,UNKNOWN,A,LOT1SW,6.0,40N,13E,3.0,COOK,79.72,1.25,99.65,FD,02/17/1841,687,191.0,4828.0,False,NaN,1
2,ABBOTT JOSEPH,UNKNOWN,NaN,NWSW,3.0,37N,12E,3.0,COOK,40.0,9.0,360.00,CN,05/09/1853,L7A,180.0,7367.0,False,2,2
3,ABBOTT SAMUEL,UNKNOWN,NaN,SW,14.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/27/1838,687,194.0,5097.0,False,NaN,3;4
4,ABBOTT SAMUEL S,UNKNOWN,NaN,SE,15.0,40N,13E,3.0,COOK,160.0,1.25,200.00,FD,11/17/1838,687,194.0,5100.0,False,NaN,3;4


In [78]:
df2.to_csv("../../data/cook_county_cleaned2.csv")